Aufgabe ist es aus der Chemotion Repository (https://www.chemotion-repository.net/welcome) alle bisher hinterlegten Substanzen zu klassifizieren und eine deskriptive Statistik des erhaltenen Datensatzes anzufertigen. Hierzu wird im Folgendem das verwendete R-Script beschrieben.
Zunächst wird ein R-Skript mit den benötigten Helfer-Funktionen eingelesen, in der benötigte Funktionen ausgelagert werden, um die Vignette übersichtlicher zu gestalten.


In [ ]:
source("../Scripts/viz-helper.R")


Die benötigten Pakete müssen geladen werden:


In [ ]:
library(readr)
library(rinchi)
library(magrittr)
library(classyfireR)
library(plyr)
library(dplyr)
library(sunburstR)
library(Rcpp)
library(rcdk)
library(ggplot2)
library(plotly)
library(metfRag)



Die canonical smiles der Substanzen aus der Chemotion Repository (https://www.chemotion-repository.net/welcome) können als XLSX-Datei heruntergeladen werden. Hierfür muss ein Account auf der Homepage der Chemotion Repository angelegt werden. Unter dem Reiter "My DB" können unter "Chemotion" alle bisher eingetragenen Substanzen markiert und dann als XLSX-Datei exportiert werden.
In der heruntergeladenen XLSX-Datei befinden sich anschließend noch Duplikate, sowie Lösungsmittel (solvents) und Reaktionspartner (reactants). Nach Entfernung dieser Substanzen wird die XLSX-Datei in eine CSV-Datei konvertiert und kann dann in R eingelesen werden.


In [ ]:
smiles <- as.character(read.csv("../Data/sample_export_16.03.2021_8.12_noDup.csv")[["canonical.smiles"]])
head(smiles,3)


Mit dem Paket "rinchi" (https://rdrr.io/github/CDK-R/rinchi/man/getinchi.html#heading-0) werden nun die canonical smiles in InChikeys konvertiert, damit diese zur Klassifizierung notwenig sind.


In [ ]:
inchikey <-sapply(smiles, get.inchi.key)


Im nächsten Schritt werden die InChiKeys verwendet, um eine Klassifizierung der Substanzen durchzuführen. Die Klassifizierung erfolgt mit dem Paket "classyfireR" (https://github.com/aberHRML/classyfireR). Die Funktion "get_classification" erzeugt ein JSON-Objekt für jede Substanz. Aus diesem Objekt können zum Beispiel Metadaten und die hierarchischen Klassifizierungsstufen ausgelesen werden. Für Substanzen die nicht klassifiziert werden konnten wird ein Objekt vom Typ "NULL" erzeugt.


In [ ]:
Classification_List <- sapply(inchikey, get_classification)
head(Classification_List,1)


Das erhaltene JSON Objekt aus ClassyfierR muss so formatiert werden, dass am Ende ein Datenframe mit der hierarischen Klassifizierung der Substanzen ausgestattet ist. Hierzu wurde eine Funktion classyfire2df() geschrieben, die als Eingabe die Classification_List und als Ausgabe einen Datenframe hat:


In [ ]:
class <- vector("character", length(Classification_List[]))
df <- classyfire2df()
df


Anzahl der zu klassifizierenden Substanzen aus der Chemotion Repository (Größe des Datensatzes):


In [ ]:
count_substances <- nrow(df)
count_substances


In dem erzeugten Dataframe stehen in den Spalten der jeweilige Klassifizierungsgrad und in den Reihen die jeweilige Substanz. Im ersten Schritt wird eine neue Spalte "classes" erzeugt, in der alle Spalten mit den Klassifizierungsgraden einer Substanz zusammengefasst werden und mit einem Semikolon getrennt werden. Im nächsten Schritt werden alle SPalten, außer die "classes"-Spalte gelöscht und die hierarisch angelegten Substanzklassen nach Anzahl aufsummiert. Klassifizierungsgrade, die in ihrem Namen einen Bindestrich haben, werden im folgenden Sunburst-Plot nicht richtig geplotted, deshalb werden die Bindestriche in Unterstriche umgewandelt.


In [ ]:
df$classes <- gsub('; NA','', paste(df$"1",df$"2",df$"3", df$"4", df$"5", df$"6", df$"7",df$"8" ,df$"9" ,sep = "; "))
df <-ddply(df,.(classes),summarize, count=length(classes) )
df <- df[-1, ]
df$classes <- gsub('-','_',df$classes)
df$classes <- gsub(';','-',df$classes)


Anzahl der erfolgreich klassifizierten Substanzen, sowie der nicht klassifizierbaren Substanzen aus der Chemotion Repository:


In [ ]:
sum(df$count)
count_substances-sum(df$count)


Mithilfe dieses Data Frames kann die Darstellung des Klassifizierungslevel "class" aller klassifizierten Substanzen prozentual als Sunburst-Plot erfolgen. Die Erstellung des Sunburst Plots erfolgte mit dem Paket "sunburstR" (https://github.com/timelyportfolio/sunburstR)


In [ ]:
sunburst(df, legend = FALSE)


Neben der Darstellung der prozentualen Anteile der Substanzklassen können mithilfe des Paketes "rcdk" (https://cran.r-project.org/web/packages/rcdk/index.html) die molaren Massen der Substanzen aus den canonical smiles berechnet werden. Der erstellte Dataframe zeigt die ersten drei Substanzen aus dem Datensatz mit der jeweiligen exakten molaren Masse [g/mol] an.


In [ ]:
substance_mass <- sapply(smiles, smiles2mass)
df_mass <- data.frame(substance_mass)
mass <- data.frame(substance= row.names(df_mass), df_mass, row.names=NULL)
head(mass,3)


Mit dem Paket "ggplot2" (https://www.rdocumentation.org/packages/ggplot2/versions/3.3.3) kann ein Histogramm der molaren Massen erstellt werden und mit dem Paket "ggplotly" (https://www.rdocumentation.org/packages/plotly/versions/4.9.3/topics/ggplotly) wird der erstellte Plot interaktiv nutzbar:


In [ ]:
histogram <- ggplot(mass,aes(substance_mass))+
  geom_histogram(binwidth=20,color="darkblue", fill="lightblue")+
  labs(title="Histogram plot: Molar mass vs count",x="Exact mass (g/mol)", y = "Count")

ggplotly(histogram)


Weiterhin kann ein hierarchischen Clusterings der Substanzen mit dem Paket 'metfRag'(https://github.com/ipb-halle/MetFragR) dargestellt werden. Diese Darstellung zeigt das Cluster von den Substanzen nach strukturellen Ähnlichkeiten:


In [ ]:
mols <- parse.smiles(smiles)
dummy <- mapply(set.property, mols, "Score", c(1:2343))
plotCluster(mols, h=0.2)


Sessioninfo:


In [ ]:
sessionInfo()